In [31]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from bs4 import BeautifulSoup
import pickle
from RecFunctions import get_products_by_type,get_products_by_problem, categorical_columns, type_column

# Functions Scrape Data and Get Info Df's For SkinStore

### *****Need to figure out how to get images in full df

In [32]:
cleansers=get_products_by_type('cleansers',14)
exfoliators=get_products_by_type('exfoliators',6)
removers=get_products_by_type('makeup-removers',3)
toners=get_products_by_type('toners',6)
mists=get_products_by_type('mists',3)
treatments=get_products_by_type('treatments',14)
serums=get_products_by_type('serums',16)
lotions=get_products_by_type('lotions',6)
moisturizers=get_products_by_type('moisturizers',23)
balms=get_products_by_type('balms',3)
oils=get_products_by_type('oils',5)
masks=get_products_by_type('masks',9)
peels=get_products_by_type('peels',3)
lips=get_products_by_type('lip-care',4)
eyes=get_products_by_type('eye-care',9)
supplements=get_products_by_type('supplements',1)
tools=get_products_by_type('tools',5)

In [33]:
acnedf,acne = get_products_by_problem('acne-blemishes',10)
agedf,age = get_products_by_problem('anti-aging',30)
darkcdf,darkcircles = get_products_by_problem('dark-circles',5)
drydf,dryness = get_products_by_problem('dry-skin',19)
ncdf,norm= get_products_by_problem('normal-combination',17)
oilydf,oily=get_products_by_problem('oily-skin',17)
sensitivedf,sensi=get_products_by_problem('sensitive-skin',17)
reddf,redness=get_products_by_problem('redness-rosacea',17)
# images=[]
# images=(reddf.image+sensitivedf.image+drydf.image+oilydf.image+ncdf.image+darkcdf.image+agedf.image+acnedf.image)

In [34]:
braa=[]
ratee=[]
pri=[]
ur=[]
imgs=[]
for i,r,p,u in zip(acnedf.prodName,acnedf.rating,acnedf.price,acnedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(agedf.prodName,agedf.rating,agedf.price,agedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(darkcdf.prodName,darkcdf.rating,darkcdf.price,darkcdf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(drydf.prodName,drydf.rating,drydf.price,drydf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(ncdf.prodName,ncdf.rating,ncdf.price,ncdf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(oilydf.prodName,oilydf.rating,oilydf.price,oilydf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(sensitivedf.prodName,sensitivedf.rating,sensitivedf.price,sensitivedf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
for i,r,p,u in zip(reddf.prodName,reddf.rating,reddf.price,reddf.url):
    braa.append(i)
    ratee.append(r)
    pri.append(p)
    ur.append(u)
#     imgs.append(im)
datadict=[]
# for i,r,p,u,img in zip(braa,ratee,pri,ur,imgs):
#     datadict.append({'prodName':i,'rating':r,'price':p,'url':u,'image':img})
for i,r,p,u in zip(braa,ratee,pri,ur):
    datadict.append({'prodName':i,'rating':r,'price':p,'url':u})
finaldf=pd.DataFrame(datadict)

In [36]:
finaldf['image']=images

In [37]:
finaldf.rating=[float(i) for i in finaldf.rating]
finaldf.price=[float(i) for i in finaldf.price]

In [38]:
see=finaldf.set_index('prodName')

In [53]:
with open('findf.pickle', 'wb') as f:
    pickle.dump(finaldf, f, pickle.HIGHEST_PROTOCOL)

## Finalize Full DF

In [39]:
categorical_columns('age',age,finaldf)
categorical_columns('darkcircles',darkcircles,finaldf)
categorical_columns('acne',acne,finaldf)
categorical_columns('dry',dryness,finaldf)
categorical_columns('redness',redness,finaldf)
categorical_columns('sensitive',sensi,finaldf)
categorical_columns('oily',oily,finaldf)
categorical_columns('normal',norm,finaldf)
categorical_columns('cleanser',cleansers,finaldf)
categorical_columns('exfoliator',exfoliators,finaldf)
categorical_columns('makeup-removers',removers,finaldf)
categorical_columns('toner',toners,finaldf)
categorical_columns('mist',mists,finaldf)
categorical_columns('treatment',treatments,finaldf)
categorical_columns('serum',serums,finaldf)
categorical_columns('lotion',lotions,finaldf)
categorical_columns('moisturizer',moisturizers,finaldf)
categorical_columns('balm',balms,finaldf)
categorical_columns('oil',oils,finaldf)
categorical_columns('mask',masks,finaldf)
categorical_columns('peel',peels,finaldf)
categorical_columns('lip',lips,finaldf)
categorical_columns('eye',eyes,finaldf)
categorical_columns('supplement',supplements,finaldf)
categorical_columns('tool',tools,finaldf)

In [40]:
with open('CategoricalItemDF.pickle', 'rb') as f:
    df = pickle.load(f)

# Get Description Info & Get Into DF

In [41]:
ingreddict=[]
for (u,p) in zip(finaldf.url,finaldf.prodName):
#     print(u,' ',p)
    response=requests.get('https://www.skinstore.com/{}.html'.format(u))
    soup=BeautifulSoup(response.content,'html.parser')
    info=soup.find('div',{'data-information-component':'ingredients'})
    brand=soup.find('div',{'data-information-component':'brand'})
    try:
        ingreddict.append({'url':u,'brand':brand.text.replace('\n',''),'ingredients':info.text.replace('\n','').split(',')})
    except:
        ingreddict.append({'url':u,'brand':brand.text.replace('\n',''),'ingredients':'NaN'})


In [43]:
ingreddf=pd.DataFrame(ingreddict)
# ingreddf.drop_duplicates(inplace=True)

In [44]:
newingred=[]
for lis in ingreddf.ingredients:
    for i in lis:
        newingred.append(i.split(','))

In [45]:
finaldf.drop_duplicates(inplace=True)

In [46]:
#ingreddf.drop_duplicates(inplace=True)

In [48]:
newfull.reset_index(inplace=True)
newfull.drop(columns=[index],inplace=True)
newfull

NameError: name 'newfull' is not defined

In [81]:
with open('IngredientsDF.pickle', 'wb') as f:
    pickle.dump(ingreddf, f, pickle.HIGHEST_PROTOCOL)

In [24]:
with open('IngredientsDF.pickle', 'rb') as f:
    idf = pickle.load(f)

In [49]:
idf.head()

,brand,ingredients,url
0,Paula's Choice,"[Aqua, Methylpropanediol (hydration), Butyle...",11174178
1,SkinCeuticals,"[Aqua / Water / Eau, Glycerin, Propylene Gly...",11289629
2,Sunday Riley,"[Opuntia Tuna Fruit (Prickly Pear) Extract, A...",10915787
3,Paula's Choice,"[Water, Glycerin, Butylene Glycol, Niacinam...",11174171
4,Dermalogica,[Key Ingredients:Quillaja Saponaria: Bark puri...,10364095


In [51]:
finaldf

,price,prodName,rating,url,image,age,darkcircles,acne,dry,redness,...,lotion,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool
0,29.99,La Roche-Posay Effaclar Adapalene Gel 0.1% Ret...,4.91,12190079,https://s1.thcdn.com/productimg/600/600/121900...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,29.50,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,https://s1.thcdn.com/productimg/600/600/111741...,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,85.00,Slip Silk Pillowcase - Queen - White,4.68,11561755,https://s1.thcdn.com/productimg/600/600/115617...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,83.00,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,3.00,11289619,https://s1.thcdn.com/productimg/600/600/112896...,1,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
4,28.00,ARCONA MEN Clarity Aftershave Pads (45 Pads),4.86,11286252,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,47.00,ARCONA Cranberry Gommage 3.4oz,5.00,11286233,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
6,85.00,Slip Silk Pillowcase - Queen - Pink,4.11,11561754,https://s1.thcdn.com/productimg/600/600/115617...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
7,38.00,ARCONA Cranberry Toner 4.05oz,4.11,11286235,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,67.00,SkinCeuticals Phyto Corrective Gel,5.00,11289629,https://s1.thcdn.com/productimg/600/600/112896...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,435.00,Dr Dennis Gross Skincare DRx FaceWare Pro Dome...,4.00,11931276,https://s1.thcdn.com/productimg/600/600/119312...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1


In [26]:
finaldf['brand']=idf['brand']

In [27]:
finaldf.head()

,price,prodName,rating,url,image,age,darkcircles,acne,dry,redness,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,brand
0,29.99,La Roche-Posay Effaclar Adapalene Gel 0.1% Ret...,4.91,12190079,https://s1.thcdn.com/productimg/600/600/121900...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Paula's Choice
1,29.50,Paula's Choice Skin Perfecting 2% BHA Liquid E...,5.00,11174178,https://s1.thcdn.com/productimg/600/600/111741...,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,SkinCeuticals
2,85.00,Slip Silk Pillowcase - Queen - White,4.68,11561755,https://s1.thcdn.com/productimg/600/600/115617...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,Sunday Riley
3,83.00,SkinCeuticals Hydrating B5 Hyaluronic Acid Gel...,3.00,11289619,https://s1.thcdn.com/productimg/600/600/112896...,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,Paula's Choice
4,28.00,ARCONA MEN Clarity Aftershave Pads (45 Pads),4.86,11286252,https://s1.thcdn.com/productimg/600/600/112862...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Dermalogica


In [28]:
# with open('FINALDFurl.pickle', 'wb') as f:
#     pickle.dump(finaldf, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
str(img).split('"//')[2][:61]

## Get Review Info & Get Into DF

In [141]:
ratingdict=[]
for u in finaldf.url:
    response=requests.get('https://www.skinstore.com/the-ordinary-aha-30-bha-2-peeling-solution-30ml/{}.html'.format(u))
    soup=BeautifulSoup(response.content,'html.parser')
    titles=soup.findAll('h3',{'class':'productReviews_topReviewTitle'})
    ratings=soup.findAll('div',{'class':'productReviews_topReviewsRatingStarsContainer'})
    contents=soup.findAll('p',{'class':'productReviews_topReviewsExcerpt'})
    dates=soup.findAll('span',{'data-js-element':'createdDate'})
    users=soup.findAll('div',{'class':'productReviews_footerDateAndName'})
    products=soup.find('div',{'data-information-component':'brand'})
    else:
        continue
    for t,r,c,d,i in zip(titles,ratings,contents,dates,users):
        ratingdict.append({'url':u,'brandName':products.text.replace('\n',''),'title':t.text.replace('\n',''),'rating':str(r).split('aria-label=')[1][1:2],'content':c.text.replace('\n','').replace('\r',''),'date':d.text,'user':i.text.replace('\n','').split('by')[1].lower()})
ratingdf=pd.DataFrame(ratingdict)
ratingdf['date']=pd.to_datetime(ratingdf.date)
ratingdf['rating']=[int(r) for r in ratingdf['rating']]
ratingdf.user=ratingdf.user.replace('','user')

In [26]:
# ratingdf.drop_duplicates(inplace=True)
ratingdf.content=[c.lower() for c in ratingdf.content]
ratingdf.shape

(3259, 7)

In [31]:
ratingdf['brandName']=[r.text.replace('\n','') for r in ratingdf['brandName']]

In [142]:
ratingdf

,brandName,content,date,image,rating,title,url,user
0,Paula's Choice,I like this product. I’ve been using it for ab...,2019-10-16,s1.thcdn.com/productimg/600/600/11174178-20346...,4,Definitely improved my skin clarity,11174178,anna
1,Paula's Choice,This product has changed my problematic skin. ...,2019-08-08,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Amazing,11174178,becky c
2,Paula's Choice,I absolutely love this product. It's one of th...,2019-05-05,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Amazing,11174178,jess
3,Paula's Choice,I was really hoping this would work for since ...,2019-04-12,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Awesome,11174178,lisa
4,Paula's Choice,Not sure why everyone doesn't use this amazing...,2018-12-13,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Extraordinary,11174178,stella2
5,Paula's Choice,I've struggled with breakouts my entire life a...,2018-12-01,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Actually Works!,11174178,carli
6,Paula's Choice,I finally understand why this has such a cult ...,2018-11-10,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Extraordinary,11174178,stella2
7,Paula's Choice,Amazing product! Clears up congested skin but ...,2018-08-24,s1.thcdn.com/productimg/600/600/11174178-20346...,5,Highly recommended!,11174178,tilly
8,Paula's Choice,"This product is literally a skin saver, that i...",2017-04-04,s1.thcdn.com/productimg/600/600/11174178-20346...,5,MUST HAVE!,11174178,anne206
9,Paula's Choice,I bought this based on the one and only review...,2015-12-23,s1.thcdn.com/productimg/600/600/11174178-20346...,5,effective,11174178,samsara33139


In [8]:
omgdf=ratingdf[['prodName','rating','url','user']]

KeyError: "['prodName'] not in index"

In [37]:
omgdf.drop_duplicates(inplace=True)

/Users/mackenziemitchell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
omgdf

,prodName,rating,url,user
0,Paula's Choice,5,11174178,becky c
1,Paula's Choice,5,11174178,jess
2,Paula's Choice,5,11174178,lisa
3,Paula's Choice,5,11174178,stella2
4,Paula's Choice,5,11174178,carli
6,Paula's Choice,5,11174178,tilly
7,Paula's Choice,5,11174178,anne206
8,Paula's Choice,5,11174178,samsara33139
9,Paula's Choice,5,11174178,suzi
10,SkinCeuticals,5,11289629,kiko


In [38]:
# with open('recdf.pickle', 'wb') as f:
#     pickle.dump(omgdf, f, pickle.HIGHEST_PROTOCOL)

In [33]:
# with open('userratingsDF.pickle', 'wb') as f:
#     pickle.dump(ratingdf, f, pickle.HIGHEST_PROTOCOL)

In [143]:
with open('newratingsDFwithimg.pickle', 'wb') as f:
    pickle.dump(ratingdf, f, pickle.HIGHEST_PROTOCOL)